In [1]:
# Import required libraries

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [2]:
# Importing the Data                                                                       Shape

Endg_Truth = loadmat('end6_groundTruth.mat')      
Ground_Truth = loadmat('groundTruth.mat')               
Urban = loadmat('Urban_R162.mat')                 
image = Urban['Y'].T                                                                #    94249*162  
groundTruth = Ground_Truth ['groundTruth']                                          #     307*307
image_shape = (Ground_Truth ['nRow'], Ground_Truth ['nCol'], image.shape[1])


def hyper_to_2D(img):
    """
    
    Converts a 3D HSI cube (m x n x p) to a 2D matrix of points (N X p)
+   N = m*n, p := number of bands

    """
    img = np.array(img)
    ndim = img.ndim

    if ndim not in [2, 3]:
      raise "invalid dimention. ndim = 2 or 3"

    if ndim == 2:
      m, n = np.shape(img)
      
      hyper_img = np.empty((m,n,1))
      hyper_img[:,:,0] = img

      return hyper_to_2D(hyper_img).ravel()

    m, n, p = np.shape(img)
    N = n * m
    
    dataset = img.T.reshape((p, N)).T

    if p == 1:
      return dataset.ravel()
    
    return dataset

image = image
groundTruth = hyper_to_2D(groundTruth)

print(image.shape)
print(groundTruth.shape)


(94249, 162)
(94249,)


In [3]:
# Standardization

image = StandardScaler().fit_transform(image)


In [4]:
# Featuer extracion with ica

from sklearn.decomposition import FastICA
image = FastICA(n_components=30,random_state=10).fit_transform(image)

print(np.shape(image))

(94249, 30)


In [5]:
# Split into train (2/3) and test (1/3) sets

from sklearn.model_selection import train_test_split
seed = 40
X_train, X_test, y_train, y_test = train_test_split(image, groundTruth, test_size = 0.30 , train_size = 0.70 , random_state = seed , stratify=groundTruth)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(65974, 30)
(28275, 30)
(65974,)
(28275,)


In [6]:
G=[0.1,1,10]
C=[0.1,1,10]
accuracy_test = []
accuracy_train = []

for c in C:
    for g in G:
    
        svm = SVC(kernel= 'rbf', random_state=1, C=c, gamma=g)
        svm.fit(X_train,y_train)
        predict_Ltest = svm.predict(X_test)
        predict_Ltrain = svm.predict(X_train)
        a1=accuracy_score(y_test,predict_Ltest)
        a2=accuracy_score(y_train,predict_Ltrain)
        accuracy_test.append([g,c,a1])
        print('g=',g,'c=',c,'accuracy test=',a1)
        accuracy_train.append([g,c,a2])
        print('g=',g,'c=',c,'accuracy train=',a2)



g= 0.1 c= 0.1 accuracy test= 0.36084880636604777
g= 0.1 c= 0.1 accuracy train= 0.3608542759268803
g= 1 c= 0.1 accuracy test= 0.36084880636604777
g= 1 c= 0.1 accuracy train= 0.3608542759268803
g= 10 c= 0.1 accuracy test= 0.5367992926613616
g= 10 c= 0.1 accuracy train= 0.5361657622699851
g= 0.1 c= 1 accuracy test= 0.36084880636604777
g= 0.1 c= 1 accuracy train= 0.3608542759268803
g= 1 c= 1 accuracy test= 0.5373651635720601
g= 1 c= 1 accuracy train= 0.5365901718858944
g= 10 c= 1 accuracy test= 0.826525198938992
g= 10 c= 1 accuracy train= 0.825037135841392
g= 0.1 c= 10 accuracy test= 0.5372590627763042
g= 0.1 c= 10 accuracy train= 0.5366204868584594
g= 1 c= 10 accuracy test= 0.8256763925729443
g= 1 c= 10 accuracy train= 0.8247643010883076
g= 10 c= 10 accuracy test= 0.8712997347480106
g= 10 c= 10 accuracy train= 0.8697214054021281
